In [ ]:
pip install binpacking geopandas

In [31]:
from h3 import h3
import json

'''
Two JSON files contain the data we need:
1. CityId-Year-ActiveSupplyCount-Map.json - this has max number of weekly unique drivers observed during a year
2. CityId-Shape-Map.json - this has a map of cityId with polygon shapes

The first one is powered by the following query:
    with year_week_city_cnt as (
      select
        year(date(datestr)) as year,
        week(date(datestr)) as week,
        city_id,
        count(distinct driver_uuid) as cnt_distinct_supplies
      from
        dwh.agg_daily_driver_segment
      where
        datestr between '{{start_date}}' and '{{end_date}}'
        and supply_hours_last7d > 0
      group by 1, 2, 3
    )
    select
      year,
      city_id,
      max(cnt_distinct_supplies) as max_weekly_supply_cnt
    from
      year_week_city_cnt
    group by 1, 2
    order by 3 desc
'''


_cityIdYearSupplyCountData = json.load(open('data/CityId-Year-ActiveSupplyCountMap.json', 'r'))
_cityIdShapeData = json.load(open('data/CityId-Shape-Map.json', 'r'))

# CityInfo holds all city specific variables
class CityInfo:
    def __init__(self, cityId, supplyCount, shape):
        self.cityId = cityId
        self.supplyCount = supplyCount
        self.shape = shape
        
# Convert _city_id_shape_data to map 
cityShapeMap = dict()
for row in _cityIdShapeData:
    cityShapeMap[str(row['city_id'])] = row['shape']

# Convert to year - cityInfo list maps
yearCityInfoListMap = dict()
for row in _cityIdYearSupplyCountData:
    year = row['year']
    cityId = row['city_id']
    supplyCount = int(row['max_weekly_supply_cnt'])
    
    # Skip cities that don't have polygon
    if cityId not in cityShapeMap:
        print(f'City {cityId} does not have associated shape')
        continue
    
    polygon = city_shape_map[cityId]
    cityInfo = CityInfo(cityId, supplyCount, cityShapeMap[cityId])
    
    if year not in yearCityInfoListMap:
        yearCityInfoListMap[year] = list()
    yearCityInfoListMap[year].append(cityInfo)

City 289 does not have associated shape
City 289 does not have associated shape
City 289 does not have associated shape
City 289 does not have associated shape
City 11 does not have associated shape
City 637 does not have associated shape
City 11 does not have associated shape
City 11 does not have associated shape
City 1635 does not have associated shape
City 1633 does not have associated shape
City 582 does not have associated shape
City 398 does not have associated shape


In [39]:
import binpacking

'''
Perform naive bin packing operation.
This step we attempt to find best bin size given relevant metric
and number of bins we wish to have.

Since the root dataset is made up of data seperated by year,
we can chose to run the entire operation on yearly basis. That will
give us a nice time representation allowing us to see if bin distribution
over the years.
'''
num_bins = 30
yearIdealBinSizeMap = dict()

for year in yearCityInfoListMap.keys():
    supplyCountList = [cityInfo.supplyCount for cityInfo in yearCityInfoListMap[year]]
    
    groups = binpacking.to_constant_bin_number(supplyCountList, num_bins)

    idealValue = sum(supplyCountList) / num_bins 
    realValues = [sum(group) for group in groups]
    yearIdealBinSizeMap[year] = idealValue

    print(f'Ideal Value per group year {year}: {idealValue},\nAllocated values: {realValues}\n')

    '''
    # Debug
    for group in groups:
        keys = list(group.keys())
    '''    

Ideal Value per group year 2019: 177622.96666666667,
Allocated values: [250088, 175125, 175125, 175125, 175125, 175125, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124, 175124]

Ideal Value per group year 2020: 139748.66666666666,
Allocated values: [212091, 137255, 137255, 137255, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254, 137254]

Ideal Value per group year 2022: 173080.36666666667,
Allocated values: [199869, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172157, 172156, 172156, 172156, 172156, 172156, 172156, 172156, 172156, 172156, 172156, 172156]

Ideal Value per group year 2021: 147320.46666666667,
Allocate

In [55]:
from collections import defaultdict, OrderedDict
import copy
import pandas as pd
import geopandas as gpd
from shapely import wkt
import folium
import json, os, random, uuid

def compute_and_print_sharding_map(cityInfoList, idealCountPerShard):
    
    # Create DataFrame
    columns = ['city_id', 'supply_count', 'shape']
    data = list()
    for cityInfo in cityInfoList:
        data.append([cityInfo.cityId, cityInfo.supplyCount, cityInfo.shape])

    # Create base panda dataframe
    df = pd.DataFrame(data, columns=columns)

    # Convert from string shape representation to shapely geometry
    df['shape'] = df['shape'].apply(wkt.loads)

    # Create geopandas frame with `shape` as geometry field
    gdf = gpd.GeoDataFrame(df, geometry='shape')
    gdf['centroid'] = gdf.centroid
    gdf['lat'] = gdf['centroid'].apply(lambda p: p.y)
    gdf['lng'] = gdf['centroid'].apply(lambda p: p.x)

    # Sort by latitude
    gdf = gdf.sort_values(by=['lat'], ascending=False)

    # Walk through sorted list and attempt to add in groups
    maxSupplyCount = idealCountPerShard

    groupsToCities = defaultdict(lambda: list())
    groupsToSupplyCount = defaultdict(lambda: 0)
    currentGroupId = 0

    for _, r in gdf.iterrows():
        # Attempt to add city to current group
        if groupsToSupplyCount[currentGroupId] + r['supply_count'] >= maxSupplyCount:
            currentGroupId = currentGroupId + 1
        # Append city to current group
        groupsToCities[currentGroupId].append(r)
        # Increment supply count
        groupsToSupplyCount[currentGroupId] = \
            groupsToSupplyCount[currentGroupId] + r['supply_count']

    print(f'Total groups: {len(groupsToCities)}')

    # Sort city_ids and print in JSON format
    json_data = OrderedDict()
    for groupId in range(len(groupsToCities)):
        cityIds = [int(city['city_id']) for city in groupsToCities[groupId]]
        cityIds.sort()
        json_data[groupId] = cityIds

    # Attempt printing heatmap in folium
    map = folium.Map(location = [15,30], tiles='Cartodb dark_matter', zoom_start = 2)

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # Walk through allocated groups and display on maps
    for groupId in range(len(groupsToCities)):
        cities = groupsToCities[groupId]
        print(f'Num cities in group {groupId}: {len(cities)}')
        color = colors[groupId % len(colors)]
        for cityGdf in cities:
            geo_j = folium.GeoJson(
                data=cityGdf['shape'], 
                name=groupId, 
                style_function=lambda x, color=color: {'color': color})
            geo_j.add_to(map)
    return json_data, map

# Driver code
result_id = uuid.uuid4()
basePath = f'results/{result_id}'
os.mkdir(basePath)

# Store all results
yearResultsMap = dict()
    
for year in yearCityInfoListMap.keys():
    jsonData, map = compute_and_print_sharding_map(yearCityInfoListMap[year], yearIdealBinSizeMap[year])
    #map.save(f'{basePath}/map-{year}.html')
    #with open(f'{basePath}/map-{year}.json', "w") as outfile:
    #    json.dump(jsonData, outfile)
    yearResultsMap[year] = jsonData
    print(f'Find results in {basePath}')
        

Total groups: 36
Num cities in group 0: 74
Num cities in group 1: 119
Num cities in group 2: 68
Num cities in group 3: 29
Num cities in group 4: 34
Num cities in group 5: 27
Num cities in group 6: 39
Num cities in group 7: 68
Num cities in group 8: 14
Num cities in group 9: 42
Num cities in group 10: 38
Num cities in group 11: 3
Num cities in group 12: 4
Num cities in group 13: 6
Num cities in group 14: 34
Num cities in group 15: 19
Num cities in group 16: 17
Num cities in group 17: 22
Num cities in group 18: 18
Num cities in group 19: 6
Num cities in group 20: 15
Num cities in group 21: 23
Num cities in group 22: 26
Num cities in group 23: 22
Num cities in group 24: 27
Num cities in group 25: 39
Num cities in group 26: 24
Num cities in group 27: 57
Num cities in group 28: 17
Num cities in group 29: 3
Num cities in group 30: 7
Num cities in group 31: 1
Num cities in group 32: 54
Num cities in group 33: 30
Num cities in group 34: 21
Num cities in group 35: 27
Find results in results/4e4

In [67]:
for year in sorted(yearResultsMap.keys()):
    currentYear = str(int(year))
    nextYear = str(int(year) + 1)
    if nextYear in yearResultsMap:
        currentYearData = yearResultsMap[currentYear]
        nextYearData = yearResultsMap[nextYear]
        
        # Walk through all groups and try to find diff
        print(f'\n\n\nDiff between {currentYear} -- {nextYear}')
        for group in currentYearData.keys():
            if group in nextYearData:
                current = currentYearData[group]
                next = nextYearData[group]
                print(f'Removed from shard {group}: {[x for x in current if x not in next]}')
                print(f'Added to shard {group}: {[x for x in next if x not in current]}\n')




Diff between 2019 -- 2020
Removed from shard 0: [37, 263, 427, 454, 536, 608, 1113, 1141, 1303, 1740]
Added to shard 0: [421, 650, 807, 1125, 1145, 1475, 1476, 1491, 1734, 1880, 2138, 2159, 2160, 2162, 2163, 2164, 2166, 2168, 2169, 2300]

Removed from shard 1: [40, 140, 292, 335, 397, 425, 491, 535, 591, 656, 1700, 1721, 1735, 1745, 2043, 2053, 2056, 2058]
Added to shard 1: [37, 263, 427, 454, 608, 1303, 1737, 1738, 1743, 1746, 1747, 2098, 2099, 2104, 2105, 2108, 2114, 2115, 2116, 2118, 2165]

Removed from shard 2: [49, 50, 285, 290, 338, 347, 349, 376, 393, 407, 492, 735, 1567, 1568, 1767, 1974, 2021, 2066, 2075, 2077]
Added to shard 2: [40, 140, 292, 335, 397, 487, 491, 535, 591, 625, 656, 1003, 1700, 1721, 1890, 1973, 2202, 2203, 2204, 2205, 2370]

Removed from shard 3: [538]
Added to shard 3: [49, 50, 285, 290, 338, 347, 349, 376, 393, 407, 492, 735, 1261, 1477, 1567, 1568, 1767, 1974, 2021, 2026, 2027, 2028, 2066, 2075, 2077, 2123, 2127, 2139, 2153, 2175, 2177, 2196, 2207, 2294